# Setup

In [1]:
import sys

sys.path.append("..")

In [2]:
# Main Constants
from pathlib import Path

# Base path for testing - adjust as needed
BASE_PATH = Path("temp").resolve()  # Points to anthropic_agent root
print(BASE_PATH)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp


# Tool Tests

## Tool: Read File

In [3]:
from anthropic_agent.common_tools.read_file import ReadFileTool

# Create the read file tool
read_file_tool = ReadFileTool(base_path=BASE_PATH)
read_file = read_file_tool.get_tool()

In [4]:
print(read_file("README.md"))

[lines 1-100 of 102 in README.md]
# anthropic-agent

A small Python library for building **Anthropic-powered agents** with:
- **streaming** responses
- **tool calling** (client tools)
- **optional server tools** (passed through to Anthropic)
- **persistence** (filesystem or Postgres) and **file storage** (local or S3)

> **For sample agent configurations examples hanve a look at agent_test.ipynb**

The primary package is `anthropic_agent` (install name: `anthropic-agent`).

## AnthropicAgent features

- **Resumable sessions**: pass `agent_uuid` to reload `messages`, `container_id`, token counters, and file registry from the DB backend.
- **Streaming output**: stream formatted chunks to an `asyncio.Queue` (`formatter="xml"` or `formatter="raw"`).
- **Tool execution loop**: executes *client tools* locally when the model returns `stop_reason == "tool_use"`, then feeds tool results back to the model.
- **Server tools passthrough**: include Anthropic server tools configs (e.g. code executio

## Tool: List Dir

In [5]:
from anthropic_agent.common_tools.list_dir import ListDirTool

list_dir_tool = ListDirTool(base_path=BASE_PATH)
list_dir = list_dir_tool.get_tool()

In [6]:
print(list_dir("."))

temp/
   - test-agent-123/
      - api_refactor_20260115_100908.md
      - API_v2_Upgrade_20260115_100908.md
      - db_migration_20260115_100908.md
      - plan_20260115_074238.md
      - plan_20260115_074249.md
   - patch_demo.md
   - README.md
   - test_file.md


## Tool: Glob File Search

In [7]:
from anthropic_agent.common_tools.glob_file_search import GlobFileSearchTool

glob_search_tool = GlobFileSearchTool(base_path=BASE_PATH)
glob_file_search = glob_search_tool.get_tool()

In [8]:
print(glob_file_search("*.md"))

patch_demo.md
test-agent-123/API_v2_Upgrade_20260115_100908.md
test-agent-123/db_migration_20260115_100908.md
test-agent-123/api_refactor_20260115_100908.md
test-agent-123/plan_20260115_074249.md
test-agent-123/plan_20260115_074238.md
README.md
test_file.md


## Tool: Grep Search

In [9]:
from anthropic_agent.common_tools.grep_search import GrepSearchTool

grep_search_tool = GrepSearchTool(base_path=BASE_PATH)
grep_search = grep_search_tool.get_tool()

In [10]:
print(grep_search("agent/|Test"))

README.md:
  7- - **persistence** (filesystem or Postgres) and **file storage** (local or S3)
  8- 
  9: > **For sample agent configurations examples hanve a look at agent_<match>test</match>.ipynb**
  10- 
  11- The primary package is `anthropic_agent` (install name: `anthropic-agent`).
  23- - **Files API + storage backends** (optional): detect generated `file_id`s, download via Anthropic Files API, and store via local filesystem or S3.
  24- 
  25: For a step-by-step diagram of the run loop, see `anthropic_<match>agent/</match>agent-flow.md`.
  26- 
  27- ## Requirements
  57- 
  58- Notes:
  59: - For local tool calling, pass `tools=[...]` (see `anthropic_<match>agent/</match>tools/sample_tools.py`).
  60- - Runs are persisted under `./data/` by default (agent config, conversation history, run logs).
  61- 
  65- 
  66- The demo app lives in `demos/fastapi_server/` and exposes:
  67: - `POST /<match>agent/</match>run`: streams output as Server-Sent Events (SSE)
  68: - `POST /<matc

## Tool: Apply Patch

In [3]:
from anthropic_agent.common_tools.apply_patch import ApplyPatchTool

apply_patch_tool = ApplyPatchTool(base_path=BASE_PATH)
apply_patch = apply_patch_tool.get_tool()

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/tools/base.py:441: UserWarning: ApplyPatchTool: Unknown docstring placeholder '"status"'. Available placeholders: ['max_patch_size_mb', 'max_file_size_mb']
  func.__doc__ = self._render_docstring()


In [4]:
from __future__ import annotations

import json
from pathlib import Path
from pprint import pprint

# We'll demonstrate ALL patch operation types on the same logical file.
# Because BASE_PATH is your "temp" folder, all paths below are relative to that.
DEMO_PATH = "patch_demo.md"
DEMO_MOVED_PATH = "patch_demo_moved.md"


def run_patch(title: str, patch_text: str, *, dry_run: bool = False) -> None:
    # print(f"\n--- {title} (dry_run={dry_run}) ---")
    pprint(json.loads(apply_patch(patch_text, dry_run=dry_run)))


# Ensure clean slate so "Add File" works
for p in [Path(BASE_PATH) / DEMO_PATH, Path(BASE_PATH) / DEMO_MOVED_PATH]:
    if p.exists():
        p.unlink()

print("Demo files prepared.")

Demo files prepared.


In [5]:
# 1) Add File
patch_add = """*** Begin Patch
*** Add File: patch_demo.md
+# Patch Demo
+
+Alpha
+Beta
+
+## Section
+- one
+- two
+
*** End Patch"""

run_patch("Add File", patch_add, dry_run=False)

{'dry_run': False,
 'hunks_applied': 0,
 'lines_added': 9,
 'lines_removed': 0,
 'op': 'add',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [6]:
# 2) Update File (simple hunk)
patch_update_simple = """*** Begin Patch
*** Update File: patch_demo.md
@@
 Alpha
-Beta
+Beta v2
*** End Patch"""

run_patch("Update File (simple)", patch_update_simple, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 1,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [7]:
# 3) Update File with a scope line (narrows matching to a subsection)
# NOTE: The scope line `@@ ## Section` anchors the search; don't repeat it as context
patch_update_scoped = """*** Begin Patch
*** Update File: patch_demo.md
@@ ## Section
 - one
-- two
+- two (updated)
*** End Patch"""

run_patch("Update File (scoped)", patch_update_scoped, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 1,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [8]:
# 4) Add a nested section so we can demo nested scopes
# Use EOF marker to append at end of file
patch_add_subsection = """*** Begin Patch
*** Update File: patch_demo.md
@@
 - two (updated)
+
+### Subsection
+note: hello
*** End of File
*** End Patch"""

run_patch("Update File (add subsection)", patch_add_subsection, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 3,
 'lines_removed': 0,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [9]:
# 5) Update File with nested scopes (class -> method style, but works for headings too)
# Nested scopes: @@ ## Section then @@ ### Subsection narrows to that subsection
patch_update_nested_scopes = """*** Begin Patch
*** Update File: patch_demo.md
@@ ## Section
@@ ### Subsection
-note: hello
+note: hello (edited)
*** End Patch"""

run_patch("Update File (nested scopes)", patch_update_nested_scopes, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 1,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [10]:
# 6) Update File targeting end-of-file (EOF marker)
# EOF marker ensures matching happens at the end of file
patch_update_eof = """*** Begin Patch
*** Update File: patch_demo.md
@@
 note: hello (edited)
+EOF line added
*** End of File
*** End Patch"""

run_patch("Update File (EOF marker)", patch_update_eof, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 0,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [11]:
# 7) Move/Rename during an update
patch_move = """*** Begin Patch
*** Update File: patch_demo.md
*** Move to: patch_demo_moved.md
@@
 # Patch Demo
+# (moved)
*** End Patch"""

run_patch("Update File + Move to", patch_move, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 0,
 'moved_from': 'patch_demo.md',
 'op': 'update',
 'path': 'patch_demo_moved.md',
 'status': 'ok'}


In [12]:
# 8) Delete File
patch_delete = """*** Begin Patch
*** Delete File: patch_demo_moved.md
*** End Patch"""

run_patch("Delete File", patch_delete, dry_run=False)

{'dry_run': False,
 'hunks_applied': 0,
 'lines_added': 0,
 'lines_removed': 14,
 'op': 'delete',
 'path': 'patch_demo_moved.md',
 'status': 'ok'}


In [13]:
# Optional: verify end state
print(list_dir("."))

NameError: name 'list_dir' is not defined

In [23]:
# Optional: inspect file content (should be gone after delete)
print(read_file("patch_demo_moved.md"))

Path does not exist: patch_demo_moved.md. Use glob_file_search to find files matching a pattern.


## Tool: Code Execution

This section demonstrates the `CodeExecutionTool`:
- Stateful execution (variables persist across calls)
- Using `embedded_tools` from within executed code
- `authorized_imports` allow/deny behavior
- Output truncation via `max_output_chars`
- Agent UUID injection patterns (manual + via `AnthropicAgent` when available)


In [24]:
from __future__ import annotations

from pathlib import Path

from anthropic_agent.common_tools.code_execution_tool import CodeExecutionTool
from anthropic_agent.tools import tool

CODE_OUTPUT_BASE = BASE_PATH / "code_tool_outputs"
CODE_OUTPUT_BASE.mkdir(parents=True, exist_ok=True)

print("CodeExecutionTool output base:", CODE_OUTPUT_BASE)


CodeExecutionTool output base: /Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs


In [25]:
# Define embedded tools callable from inside code

@tool
def add(a: float, b: float) -> str:
    """Add two numbers.

    Args:
        a: First number
        b: Second number
    """
    return str(a + b)


@tool
def to_upper(s: str) -> str:
    """Upper-case a string.

    Args:
        s: Input string
    """
    return s.upper()


In [26]:
# Instantiate CodeExecutionTool with embedded_tools + authorized_imports
# - json is NOT in the default BASE_BUILTIN_MODULES list, so we explicitly allow it.

code_tool = CodeExecutionTool(
    output_base_path=CODE_OUTPUT_BASE,
    embedded_tools=[add, to_upper],
    authorized_imports=["json"],
    max_output_chars=2_000,
)

# In real agent usage, the UUID is injected automatically.
# In a notebook, we set it manually.
code_tool.set_agent_uuid("notebook-demo-agent")

code_execute = code_tool.get_tool()

print("Tool name:", code_execute.__tool_schema__["name"])
print("Has __tool_instance__:", hasattr(code_execute, "__tool_instance__"))
print("Injected UUID:", code_tool.agent_uuid)


Tool name: code_execution
Has __tool_instance__: True
Injected UUID: notebook-demo-agent


In [27]:
# Scenario 1: Stateful execution (variables persist)
print(code_execute("""
x = 10
print('x initially:', x)
"""))

print(code_execute("""
# x is still defined here
x = x + 5
print('x after increment:', x)
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_123736_1e78dc8f.txt

x initially: 10

output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_123736_9406f79a.txt

x after increment: 15



In [28]:
# Scenario 2: Call embedded tools from executed code
print(code_execute("""
print('add(2, 3) =', add(2, 3))
print('to_upper(hello) =', to_upper('hello'))
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_123753_e0a42c7d.txt

add(2, 3) = 5
to_upper(hello) = HELLO



In [ ]:
# Scenario 3: Authorized import allowed (json was explicitly whitelisted)
print(code_execute("""
import json
payload = {"ok": True, "n": 123}
print(json.dumps(payload, sort_keys=True))
"""))


In [ ]:
# Scenario 3: Authorized import allowed (json was explicitly whitelisted)
print(code_execute("""
import json
payload = {"ok": True, "n": 123}
print(json.dumps(payload, sort_keys=True))
"""))


In [ ]:
# Scenario 3: Authorized import allowed (json was explicitly whitelisted)
print(code_execute("""
import json
payload = {"ok": True, "n": 123}
print(json.dumps(payload, sort_keys=True))
"""))


In [ ]:
# Scenario 3: Authorized import allowed (json was explicitly whitelisted)
print(code_execute("""
import json
payload = {"ok": True, "n": 123}
print(json.dumps(payload, sort_keys=True))
"""))


In [29]:
# Scenario 3: Authorized import allowed (json was explicitly whitelisted)
print(code_execute("""
import json
payload = {"ok": True, "n": 123}
print(json.dumps(payload, sort_keys=True))
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_123756_fa7a2d81.txt

{"n": 123, "ok": true}



In [30]:
# Scenario 4: Unauthorized import blocked (os is not allowed)
print(code_execute("""
import os
print('this should not run')
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_123756_1561deae.txt

[Execution Error]: Code execution failed at line 'import os' due to InterpreterError: Import of os is not allowed. Authorized imports are: ['math', 'queue', 'datetime', 're', 'itertools', 'json', 'stat', 'random', 'unicodedata', 'collections', 'statistics', 'time']


In [31]:
# Scenario 5: Output truncation (small max_output_chars)
small_output_tool = CodeExecutionTool(
    output_base_path=CODE_OUTPUT_BASE,
    embedded_tools=[],
    authorized_imports=[],
    max_output_chars=300,
)
small_output_tool.set_agent_uuid("notebook-demo-agent")
small_output = small_output_tool.get_tool()

print(small_output("""
print('A' * 2000)
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_123813_91ceac83.txt


... [truncated, showing last 300 chars] ...
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
..._This content has been truncated to stay below 300 characters_...
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA



In [32]:
# Scenario 6: Missing UUID (shows warning and does not write a file)
no_uuid_tool = CodeExecutionTool(
    output_base_path=CODE_OUTPUT_BASE,
    embedded_tools=[add],
    authorized_imports=[],
    max_output_chars=1_000,
)
no_uuid_execute = no_uuid_tool.get_tool()

print(no_uuid_execute("""
print('Running without agent_uuid set')
print('add(1, 2) =', add(1, 2))
"""))


[Warning: agent_uuid is not set. The agent must inject the UUID via set_agent_uuid() before code execution can write output files.]

Running without agent_uuid set
add(1, 2) = 3



In [33]:
# Scenario 7: Agent UUID injection (no model call required)
# This demonstrates the "duck-typed" protocol: the agent sets UUID on tool instances.

try:
    from anthropic_agent import AnthropicAgent

    injected_tool = CodeExecutionTool(
        output_base_path=CODE_OUTPUT_BASE,
        embedded_tools=[add],
        authorized_imports=[],
        max_output_chars=1_000,
    )
    injected_fn = injected_tool.get_tool()

    agent = AnthropicAgent(
        system_prompt="UUID injection demo (no run)",
        tools=[injected_fn],
    )

    print("Agent UUID:", agent.agent_uuid)
    print("Tool UUID:", injected_tool.agent_uuid)
    print("UUIDs match:", agent.agent_uuid == injected_tool.agent_uuid)

except Exception as e:
    # If the notebook env doesn't have the Anthropic dependency installed,
    # importing AnthropicAgent may fail.
    print("Skipping AnthropicAgent injection demo:", type(e).__name__, str(e))


Agent UUID: 035c0214-ea79-421f-9471-bdfb136febf6
Tool UUID: 035c0214-ea79-421f-9471-bdfb136febf6
UUIDs match: True


In [34]:
# Build a CodeExecutionTool that embeds other common_tools
# Note: these tool functions were created earlier in this notebook.

code_tool_common_tools = CodeExecutionTool(
    output_base_path=CODE_OUTPUT_BASE,
    embedded_tools=[
        read_file,
        list_dir,
        glob_file_search,
        grep_search,
        apply_patch,
    ],
    authorized_imports=["json"],
    max_output_chars=2_000,
)
code_tool_common_tools.set_agent_uuid("notebook-demo-agent")
code_execute_common_tools = code_tool_common_tools.get_tool()

print("Embedded tool names:", [t.__tool_schema__["name"] for t in code_tool_common_tools.embedded_tools])

Embedded tool names: ['read_file', 'list_dir', 'glob_file_search', 'grep_search', 'apply_patch']


In [35]:
# Example A: Call list_dir from within executed code
print(code_execute_common_tools("""
result = list_dir(".")
print("Directory listing from code:")
print(result)
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124836_b8617e46.txt

Directory listing from code:
temp/
   - test-agent-123/
      - api_refactor_20260115_100908.md
      - API_v2_Upgrade_20260115_100908.md
      - db_migration_20260115_100908.md
      - plan_20260115_074238.md
      - plan_20260115_074249.md
   - README.md
   - test_file.md



In [36]:
# Example B: Call glob_file_search from within executed code
print(code_execute_common_tools("""
files = glob_file_search("*.md")
print("Markdown files found:")
print(files)
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124841_0726d966.txt

Markdown files found:
test-agent-123/API_v2_Upgrade_20260115_100908.md
test-agent-123/db_migration_20260115_100908.md
test-agent-123/api_refactor_20260115_100908.md
test-agent-123/plan_20260115_074249.md
test-agent-123/plan_20260115_074238.md
README.md
test_file.md



In [37]:
# Example C: Call read_file from within executed code
print(code_execute_common_tools("""
content = read_file("README.md", start_line_one_indexed=1, no_of_lines_to_read=10)
print("First 10 lines of README.md:")
print(content)
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124848_dcbfe4f9.txt

First 10 lines of README.md:
[lines 1-10 of 102 in README.md]
# anthropic-agent

A small Python library for building **Anthropic-powered agents** with:
- **streaming** responses
- **tool calling** (client tools)
- **optional server tools** (passed through to Anthropic)
- **persistence** (filesystem or Postgres) and **file storage** (local or S3)

> **For sample agent configurations examples hanve a look at agent_test.ipynb**





In [38]:
# Example D: Call grep_search from within executed code
print(code_execute_common_tools("""
matches = grep_search("agent")
print("Grep results for 'agent':")
print(matches[:500])  # truncate for display
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124848_2c4a4e4d.txt

Grep results for 'agent':
README.md:
  1: # anthropic-<match>agent</match>
  2- 
  3: A small Python library for building **Anthropic-powered <match>agent</match>s** with:
  4- - **streaming** responses
  5- - **tool calling** (client tools)
  7- - **persistence** (filesystem or Postgres) and **file storage** (local or S3)
  8- 
  9: > **For sample <match>agent</match> configurations examples hanve a look at <match>agent</match>_test.ipynb**
  10- 
  11: The primary package is `anthropic_<match>agent</match>` (install na



In [39]:
# Example E: Call apply_patch from within executed code (create a file)
print(code_execute_common_tools('''
import json

patch_text = """*** Begin Patch
*** Add File: code_generated.md
+# Generated from Code
+
+This file was created by calling apply_patch from inside CodeExecutionTool.
*** End Patch"""

result = apply_patch(patch_text, dry_run=False)
print("apply_patch result:")
print(json.dumps(json.loads(result), indent=2))
'''))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124849_7ac1efd6.txt

apply_patch result:
{
  "status": "ok",
  "op": "add",
  "path": "code_generated.md",
  "hunks_applied": 0,
  "lines_added": 3,
  "lines_removed": 0,
  "dry_run": false
}



In [40]:
# Example F: Verify the file was created, then read it
print(code_execute_common_tools("""
# Verify file exists
files = glob_file_search("code_generated.md")
print("File exists:", "code_generated.md" in files)

# Read the file
content = read_file("code_generated.md")
print("File content:")
print(content)
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124850_06290c98.txt

File exists: True
File content:
[lines 1-3 of 3 in code_generated.md]
# Generated from Code

This file was created by calling apply_patch from inside CodeExecutionTool.



In [41]:
# Example G: Chained workflow - search, read, and summarize in one code block
print(code_execute_common_tools("""
# 1. Find all markdown files
md_files = glob_file_search("*.md")
file_list = [f.strip() for f in md_files.strip().split("\\n") if f.strip()]
print(f"Found {len(file_list)} markdown files")

# 2. Read first 5 lines of each (up to 3 files)
for fname in file_list[:3]:
    print(f"\\n--- {fname} ---")
    content = read_file(fname, start_line_one_indexed=1, no_of_lines_to_read=5)
    # Skip header line
    lines = content.split("\\n")[1:6]
    print("\\n".join(lines))
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124852_693f170c.txt

Found 8 markdown files

--- code_generated.md ---
# Generated from Code

This file was created by calling apply_patch from inside CodeExecutionTool.

--- test-agent-123/API_v2_Upgrade_20260115_100908.md ---
# API v2 Upgrade

Upgrade notes here.


--- test-agent-123/db_migration_20260115_100908.md ---
# Database Migration Plan

## Overview
Migrate from SQLite to PostgreSQL for production.


[Last value]: ['# Database Migration Plan', '', '## Overview', 'Migrate from SQLite to PostgreSQL for production.', '']


In [42]:
# Cleanup: delete the generated file
print(code_execute_common_tools('''
import json

patch_text = """*** Begin Patch
*** Delete File: code_generated.md
*** End Patch"""

result = apply_patch(patch_text, dry_run=False)
print("Cleanup result:")
print(json.dumps(json.loads(result), indent=2))
'''))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124853_84d110d4.txt

Cleanup result:
{
  "status": "ok",
  "op": "delete",
  "path": "code_generated.md",
  "hunks_applied": 0,
  "lines_added": 0,
  "lines_removed": 3,
  "dry_run": false
}

